In [ ]:

from scipy import signal
from scipy.integrate import simps
import numpy as np
from fooof import FOOOF
import pickle

def harmonics_removal(signal, fs, harmonics, dftbandwidth=1, dftneighbourwidth=2):
    """
    Removes beta harmonics in a signal via spectrum interpolation.

    Parameters:
    - signal: 1D numpy array, the input signal
    - fs: float, sampling rate in Hz
    - harmonics: list of floats, harmonics frequencies in Hz (e.g., [50, 100, 150])
    - dftbandwidth: float, half bandwidth of harmonics frequency bands in Hz (default 1)
    - dftneighbourwidth: float, width of neighbouring frequencies in Hz (default 2)

    Returns:
    - cleaned_signal: 1D numpy array, the signal withouth the indicated beta harmonics    """
    # FFT of the signal
    N = len(signal)
    freqs = np.fft.fftfreq(N, 1/fs)
    signal_fft = np.fft.fft(signal)

    # Helper function to get indices of frequency bins
    def get_freq_indices(freq, bandwidth, fs, N):
        
        return np.where((freqs >= (freq - bandwidth)) & (freqs <= (freq + bandwidth)))[0]

    # Process each harmonic
    for f in harmonics:
        harmonics_indices = get_freq_indices(f, dftbandwidth, fs, N)
       
        harmonics_indices = np.concatenate((harmonics_indices, get_freq_indices(-f, dftbandwidth, fs, N)))
       
        for harmonics_index in harmonics_indices:
            # Find neighbouring indices
            lower_bound = f - dftneighbourwidth - dftbandwidth
            upper_bound = f + dftneighbourwidth + dftbandwidth
            neighbours = np.where((freqs >= lower_bound) & (freqs <= upper_bound) & 
                                  ((freqs < (f - dftbandwidth)) | (freqs > (f + dftbandwidth))))[0]
            
            # Compute the mean amplitude of neighbouring bins
            if len(neighbours) > 1:
                neighbour_freqs = freqs[neighbours]
          
            
                neighbour_amplitudes = np.abs(signal_fft[neighbours])
                
                interpolated_amplitude = np.mean(neighbour_amplitudes)
                original_phase = np.angle(signal_fft[harmonics_index])
                # Replace the amplitude of the harmonics frequency bin by the interpolated value
                signal_fft[harmonics_index] = interpolated_amplitude * np.exp(1j * original_phase)
    # Inverse FFT to get the cleaned signal

    cleaned_signal = np.fft.ifft(signal_fft).real
    
    return cleaned_signal

#Center of gravity method for computing central frequency
def  cog(f,pxx,f1,f2):
    prod=f*pxx
    cog=abs(simps(prod[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)])/simps(pxx[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)]))
    return cog

#Aperiodic function that will be fitted to the power spectral density and removed for the computation of the spectral power

def aper(f,offset,exp):
     return 10**offset/(f**exp)


  
def compute_power(data, n_pop, D_d, fs=1000, fmin=50,fmax=120, nparseg=1000):
    """
    Compute gamma and beta spectral power and their thresholds from neuronal activity data.

    Parameters:
    - data (list): activity of the nucleus
    - n_pop (int): Number of neurons in the population.
    - fs (float): Sampling frequency (Hz).
    - fmin (float): Minimum frequency for gamma band (Hz).
    - fmax (float): Maximum frequency for gamma band (Hz).
    - nparseg (int): Number of samples per segment for Welch's method.

    Returns:
    - gamma_power (float): gamma spectral power.
    - threshold_gamma (float): Threshold for gamma/beta (the threshold does not depend on the band considered) power based on Poisson proecess
    - beta_power (float): beta spectral power.
    """
 


    f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg/2),nfft=max(30000,nparseg), scaling='density', window='hamming') 
   
      
    f_beta=cog(f,pxx,10,30)
    
    if float(D_d)>0.9:
        harmonics=np.arange(2,5)*f_beta 
        data=harmonics_removal(data, fs, harmonics, 5, 3)
        f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg/2),nfft=max(30000,nparseg), scaling='density', window='hamming') 
     
    fm = FOOOF(max_n_peaks=2,peak_width_limits=[2, 50],verbose=False)
    fm.fit(f, pxx, freq_range=[5,500])    

    pxx[f>5]=pxx[f>5]-aper(f[f>5],*fm.aperiodic_params_)
      
   
      
    gamma_power= simps(pxx[(fmin<f) & (f<fmax)], f[(fmin<f) & (f<fmax)])
    gamma_power = gamma_power/(fmax-fmin)

    fmin_beta=10
    fmax_beta=30
    beta_power = simps(pxx[(fmin_beta<f) & (f<fmax_beta)], f[(fmin_beta<f) & (f<fmax_beta)])
    beta_power = beta_power/(fmax_beta-fmin_beta)
      
    
      

    gamma_power_pois=[]
    beta_power_pois=[]
    #lower the number if iterations if you want faster analysis
    for n in range(100):
          pois=np.zeros(len(data))
          #generation of binomial process equivalent to the activity considered
          pois=np.random.binomial(n_pop,np.mean(data)/n_pop,len(data))
          #power spectral density of poissonian activity
          f_poisson,Pxx__poisson = signal.welch(pois, fs, nperseg=nparseg, noverlap=int(nparseg/2),nfft=max(30000,nparseg), scaling='density', window='hamming')
          #mean power of poissonian activity
          pw = simps(Pxx__poisson[(fmin<f) & (f<fmax)], f[(fmin<f) & (f<fmax)])
          pw = pw/(fmax-fmin)
          gamma_power_pois.append(pw)
          pw = simps(Pxx__poisson[(fmin_beta<f) & (f<fmax_beta)], f[(fmin_beta<f) & (f<fmax_beta)])
          pw = pw/(fmax_beta-fmin_beta)
          beta_power_pois.append(pw)
    treshold_gamma=np.mean(np.array(gamma_power_pois))
   
    return gamma_power, treshold_gamma,beta_power


# Power spectral densities of all nuclei, for healthy and PD conditions

In [ ]:
from cycler import cycler
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.integrate import simps
import numpy as np
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
# Set figure and style configurations
fig = plt.figure(figsize=(24, 14))
color_cycler = cycler(color=['blue', 'red'])
plt.rc('axes', prop_cycle=color_cycler)
result = pickle.load(open("dd.p", "rb"))
# Iterate through results
for D_d in result:
    if D_d !='0.75' and  D_d !='1.0':
        continue

   
    n_trials = 5
    fs = 1000  # Sampling frequency
    t_s = 1    # Time step
    nparseg = int(1000 / t_s)  # Welch segment length

    # Initialize arrays to store PSD data
    Pxx_den_STN_final, Pxx_den_GPe_TA_final, Pxx_den_GPe_TI_final = [], [], []
    Pxx_den_FSN_final, Pxx_den_D2_final, Pxx_den_D1_final = [], [], []

    # Compute PSDs for each trial
    for i in range(n_trials):
        data_D1, data_D2, data_FSN, data_GPe_TA, data_GPe_TI, data_STN = result[D_d][i]

        def compute_psd(data, freq_range=(1, 200)):
            """Compute PSD using Welch's method for a given frequency range."""
            f, Pxx_den = welch(
                data, fs, nperseg=nparseg, noverlap=nparseg // 2,
                nfft=max(30000, nparseg), scaling='density', window='hamming'
            )
            return f[(f > freq_range[0]) & (f < freq_range[1])], Pxx_den[(f > freq_range[0]) & (f < freq_range[1])]

        f_STN, Pxx_den_STN = compute_psd(data_STN)
        f_GPe_TA, Pxx_den_GPe_TA = compute_psd(data_GPe_TA)
        f_GPe_TI, Pxx_den_GPe_TI = compute_psd(data_GPe_TI)
        f_FSN, Pxx_den_FSN = compute_psd(data_FSN)
        f_D2, Pxx_den_D2 = compute_psd(data_D2)
        f_D1, Pxx_den_D1 = compute_psd(data_D1)

        Pxx_den_STN_final.append(Pxx_den_STN)
        Pxx_den_GPe_TA_final.append(Pxx_den_GPe_TA)
        Pxx_den_GPe_TI_final.append(Pxx_den_GPe_TI)
        Pxx_den_FSN_final.append(Pxx_den_FSN)
        Pxx_den_D2_final.append(Pxx_den_D2)
        Pxx_den_D1_final.append(Pxx_den_D1)

    # Convert final PSD arrays to numpy arrays
    Pxx_den_arrays = {
        'STN': np.array(Pxx_den_STN_final),
        'GPe-TA': np.array(Pxx_den_GPe_TA_final),
        'GPe-TI': np.array(Pxx_den_GPe_TI_final),
        'FSN': np.array(Pxx_den_FSN_final),
        'D2': np.array(Pxx_den_D2_final),
        'D1': np.array(Pxx_den_D1_final)
    }

    # Define plot settings
    y_range = [10**(-3.5), 10**3]
    f_range = [0, 200]
    label = 'Healthy' if D_d=='0.75' else 'PD'
    lw = 4

    # Plot each nucleus
    titles = ['STN', 'D2', 'D1', 'GPe-TA', 'GPe-TI', 'FSN']
    
    for idx, key in enumerate(titles,1):
        plt.subplot(2, 3, idx)
        plt.yscale('log')
        plt.plot(f_STN, np.mean(Pxx_den_arrays[key], axis=0), label=label, linewidth=lw)
        plt.fill_between(
            f_STN,
            np.mean(Pxx_den_arrays[key], axis=0) - np.std(Pxx_den_arrays[key], axis=0),
            np.mean(Pxx_den_arrays[key], axis=0) + np.std(Pxx_den_arrays[key], axis=0),
            alpha=0.5
        )
        plt.title(f'{key} ', fontsize=32)
        plt.xticks([0, 50, 100, 150, 200])
        plt.xlim(f_range)
        plt.ylim(y_range)
        if idx > 3:
            plt.xlabel('Frequency [Hz]', labelpad=12)
        if idx in [1, 4]:
            plt.ylabel('PSD [a.u.]', labelpad=12)
        plt.legend(loc='upper right', framealpha=0)

    plt.tight_layout()


plt.show()


# Gamma and beta power across Dd

Computing the gamma and beta power for all nuclei, across Dd

In [ ]:
import numpy as np
import pickle

# Initialize arrays to store results
gamma_means = []  # Mean gamma power for each nucleus across dd
gamma_stds = []   # Standard deviation of gamma power for each nucleus across dd
beta_means = []  # Mean gamma power for each nucleus across dd
beta_stds = []   # Standard deviation of beta power for each nucleus across dd
gamma_thresholds = []  # Threshold power for each nucleus across dd
dd_values = []  # Stores the dd values

nuclei = ['D1', 'D2', 'FSN', 'GPe-TA', 'GPe-TI', 'STN']  # List of nuclei to process
n_pop = [6000, 6000, 420, 264, 780, 408]  # Number of neurons in the population

# Load result dataset
result = pickle.load(open("dd.p", "rb"))



# Iterate through dd in the result dataset
for D_d in result:

        print(D_d)
        # Arrays to store gamma power for each nucleus in this D_d
        dd_gamma_powers = {nucleus: [] for nucleus in nuclei}
        dd_gamma_powers_thresh= {nucleus: [] for nucleus in nuclei}
        dd_beta_powers = {nucleus: [] for nucleus in nuclei}
        for i in range(len(result[D_d])):  
            fr_D1, fr_D2, fr_FSN, fr_GPe_TA, fr_GPe_TI, fr_STN = result[D_d][i]
            firing_rates = [fr_D1, fr_D2, fr_FSN, fr_GPe_TA, fr_GPe_TI, fr_STN]
            
            # Compute gamma power for each nucleus
            for nucleus, rates, pop_size in zip(nuclei, firing_rates, n_pop):

                gamma_power, gamma_thresh,beta_power = compute_power(rates,pop_size,float(D_d))
                dd_gamma_powers[nucleus].append(gamma_power)
                dd_beta_powers[nucleus].append(beta_power)
                dd_gamma_powers_thresh[nucleus].append(gamma_thresh)
        
        # Compute statistics for this D_d across trials
        dd_means = []
        dd_stds = []
        dd_means_beta = []
        dd_stds_beta = []
        dd_thresholds = [] 

        for nucleus in nuclei:
            mean_power = np.mean(dd_gamma_powers[nucleus])
            std_power = np.std(dd_gamma_powers[nucleus])
            mean_power_beta= np.mean(dd_beta_powers[nucleus])
            std_power_beta = np.std(dd_beta_powers[nucleus])
            threshold_power =   np.mean(dd_gamma_powers_thresh[nucleus]) 
            dd_means.append(mean_power)
            dd_stds.append(std_power)
            dd_means_beta.append(mean_power_beta)
            dd_stds_beta.append(std_power_beta)
            dd_thresholds.append(threshold_power)
        
        # Append results to overall lists
        gamma_means.append(dd_means)
        gamma_stds.append(dd_stds)
        beta_means.append(dd_means_beta)
        beta_stds.append(dd_stds_beta)
        gamma_thresholds.append(dd_thresholds)
        dd_values.append(D_d)


Plotting the gamma and beta power for all nuclei, across Dd

In [ ]:
import matplotlib.pyplot as plt

# Nuclei names for labeling
nuclei = ['D1', 'D2', 'FSN', 'GPe-TA', 'GPe-TI', 'STN']

plt.rcParams.update({'font.size': 25})
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Loop through each nucleus and plot the results
for i, nucleus in enumerate(nuclei):
    row = i // 3  # Determine the row for the subplot (0 or 1)
    col = i % 3   # Determine the column for the subplot (0, 1, or 2)

 
    plt.subplot(2, 3, i + 1)  

    myblue = [0, 0.5, 1]  

    plt.errorbar([float(value) for value in dd_values], 
                 [gamma_means[j][i] for j in range(len(gamma_means))],
                 yerr=[gamma_stds[j][i] for j in range(len(gamma_stds))],
                 label='Gamma Power', linestyle='-', color=myblue, markersize=8, capsize=5, elinewidth=2,marker='.')

    
    plt.errorbar([float(value) for value in dd_values], 
                 [gamma_thresholds[j][i] for j in range(len(gamma_thresholds))],
                  linestyle='--', color='black', linewidth=3, markersize=8)
    plt.errorbar([float(value) for value in dd_values], 
                 [beta_means[j][i] for j in range(len(beta_means))],
                 yerr=[beta_stds[j][i] for j in range(len(beta_stds))],
                 label='Beta Power', linestyle='-', color='red', markersize=8, capsize=5, elinewidth=2,marker='.')
    

  
    plt.title(f'{nucleus}')
    if i>2:
        plt.xlabel('Dopamine depletion')
    if i==0 or i==3:
        plt.ylabel('Spectral power')

    plt.yscale('log')  
    plt.ylim(10**(-4),10**2)
    plt.legend(framealpha=0)


plt.tight_layout()
plt.show()
